<a href="https://www.kaggle.com/code/averma111/pytorch-crabage-s3e16?scriptVersionId=131530382" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import torch
from tqdm.notebook import tqdm
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings('ignore')
seed = 42
import os
torch.manual_seed(seed)
np.random.seed(seed)

sns.set_style("darkgrid")
pd.set_option('mode.chained_assignment',None)

In [2]:
class Acquisition:
    
    def __init__(self,root_path):
        self.root_path = root_path
        
    def get_dataframe(self,filename):
        return pd.read_csv(os.path.join(self.root_path,filename))
    
    
    
acq = Acquisition(root_path='/kaggle/input/playground-series-s3e16')
acq_original = Acquisition(root_path='/kaggle/input/crab-age-prediction')

In [3]:
train = acq.get_dataframe(filename='train.csv')
train.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [4]:
original = acq_original.get_dataframe(filename='CrabAgePrediction.csv')
original.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,F,1.4375,1.1750,0.4125,24.635715,12.332033,5.584852,6.747181,9
1,M,0.8875,0.6500,0.2125,5.400580,2.296310,1.374951,1.559222,6
2,I,1.0375,0.7750,0.2500,7.952035,3.231843,1.601747,2.764076,6
3,F,1.1750,0.8875,0.2500,13.480187,4.748541,2.282135,5.244657,10
4,I,0.8875,0.6625,0.2125,6.903103,3.458639,1.488349,1.700970,6


In [5]:
def summary(text, df):
    print(f'{text} shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summary['null'] = df.isnull().sum()
    summary['unique'] = df.nunique()
    summary['min'] = df.min()
    summary['median'] = df.median()
    summary['max'] = df.max()
    summary['mean'] = df.mean()
    summary['std'] = df.std()
    summary['duplicate'] = df.duplicated().sum()
    return summary

In [6]:
summary('train',train)

train shape: (74051, 10)


,dtypes,null,unique,min,median,max,mean,std,duplicate
id,int64,0,74051,0,37025.000000,74050,37025.000000,21376.826729,0
Sex,object,0,3,F,NaN,M,NaN,NaN,0
Length,float64,0,144,0.1875,1.375000,2.012815,1.317460,0.287757,0
Diameter,float64,0,122,0.1375,1.075000,1.6125,1.024496,0.237396,0
Height,float64,0,65,0.0,0.362500,2.825,0.348089,0.092034,0
Weight,float64,0,3096,0.056699,23.799405,80.101512,23.385217,12.648153,0
Shucked Weight,float64,0,1766,0.028349,9.908150,42.184056,10.104270,5.618025,0
Viscera Weight,float64,0,967,0.042524,4.989512,21.54562,5.058386,2.792729,0
Shell Weight,float64,0,1048,0.042524,6.931453,28.491248,6.723870,3.584372,0
Age,int64,0,28,1,10.000000,29,9.967806,3.175189,0


In [7]:
summary('original',original)

original shape: (3893, 9)


,dtypes,null,unique,min,median,max,mean,std,duplicate
Sex,object,0,3,F,NaN,M,NaN,NaN,0
Length,float64,0,134,0.1875,1.362500,2.0375,1.311306,0.300431,0
Diameter,float64,0,111,0.1375,1.062500,1.625,1.020893,0.248233,0
Height,float64,0,51,0.0,0.362500,2.825,0.349374,0.104976,0
Weight,float64,0,2343,0.056699,22.792998,80.101512,23.567275,13.891201,0
Shucked Weight,float64,0,1482,0.028349,9.539607,42.184056,10.207342,6.275275,0
Viscera Weight,float64,0,867,0.014175,4.861939,21.54562,5.136546,3.104133,0
Shell Weight,float64,0,907,0.042524,6.662133,28.491248,6.795844,3.943392,0
Age,int64,0,28,1,10.000000,29,9.954791,3.220967,0
